# **Real Time Data Processing**

In [1]:
#import reqired libraries
import math
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
real_time_data = pd.read_csv("../../Raw GPS data Kandy Buses/digana_2022_10.csv")

In [3]:
real_time_data.shape

(978917, 6)

In [4]:
real_time_data.isnull().sum()

id            0
deviceid      0
devicetime    0
latitude      0
longitude     0
speed         0
dtype: int64

In [5]:
real_time_data.head()

,id,deviceid,devicetime,latitude,longitude,speed
0,1453607164,116,2022-09-30 23:56:32,8.222097,80.522697,0.0000
1,1453607165,116,2022-09-30 23:56:47,8.222097,80.522697,0.0000
2,1453607166,116,2022-09-30 23:57:02,8.222097,80.522697,0.0000
3,1453607167,116,2022-09-30 23:57:13,8.221493,80.522667,16.1987
4,1453607168,116,2022-09-30 23:57:28,8.219942,80.522738,24.2981


# Data Preprocessing

In [6]:
real_time_data.sort_values(by='devicetime', inplace=True)
real_time_data = real_time_data[real_time_data['latitude'] != 0]
real_time_data = real_time_data[real_time_data['longitude'] != 0]
real_time_data = real_time_data[real_time_data['devicetime'] >=  '2022-10-01']
real_time_data = real_time_data[real_time_data['devicetime'] <= '2022-10-07']
real_time_data.drop(columns=['id'], inplace=True)
real_time_data.reset_index(drop = True, inplace = True)

In [7]:
real_time_data.head()

,deviceid,devicetime,latitude,longitude,speed
0,116,2022-10-01 00:00:13,8.195475,80.525735,26.9979
1,116,2022-10-01 00:00:28,8.193405,80.525917,30.2376
2,116,2022-10-01 00:00:43,8.191373,80.526890,31.8575
3,116,2022-10-01 00:00:58,8.189095,80.527400,34.5572
4,116,2022-10-01 00:01:13,8.186705,80.527848,35.0972


In [8]:
# #Function finds whether the bus is inside the bus stations or not
# def isOutsideBusStation(lat, lon):
    
#     lat = math.radians(lat)
#     lon = math.radians(lon)

#     lat_Kandy = math.radians(7.292462226)
#     lon_Kandy = math.radians(80.6349778)

#     lat_Digana = math.radians(7.29896)
#     lon_Digana = math.radians(80.73472)

#     delta_lat1 = lat_Kandy - lat
#     delta_lon1 = lon_Kandy - lon

#     delta_lat2 = lat_Digana - lat
#     delta_lon2 = lon_Digana - lon

#     # Haversine formula
#     a1 = math.sin(delta_lat1/2)**2 + math.cos(lat) * math.cos(lat_Kandy) * math.sin(delta_lon1/2)**2
#     c1 = 2 * math.atan2(math.sqrt(a1), math.sqrt(1-a1))

#     a2 = math.sin(delta_lat2/2)**2 + math.cos(lat) * math.cos(lat_Digana) * math.sin(delta_lon2/2)**2
#     c2 = 2 * math.atan2(math.sqrt(a2), math.sqrt(1-a2))

#     # Radius of the Earth in kilometers (mean value)
#     R = 6371.0

#     # Calculate the distance
#     distance1 = R * c1 * 1000
#     distance2 = R * c2 * 1000

#     if distance1 < 150 or distance2 < 150:
#         return False
#     return True

In [9]:
processed_data = real_time_data.copy()

In [10]:
# Convert devicetime column to datetime format
processed_data['devicetime'] = pd.to_datetime(processed_data['devicetime'])

processed_data['date'] = processed_data['devicetime'].dt.date
processed_data['time'] = processed_data['devicetime'].dt.time
processed_data = processed_data.drop("devicetime", axis=1)
processed_data.head()

,deviceid,latitude,longitude,speed,date,time
0,116,8.195475,80.525735,26.9979,2022-10-01,00:00:13
1,116,8.193405,80.525917,30.2376,2022-10-01,00:00:28
2,116,8.191373,80.526890,31.8575,2022-10-01,00:00:43
3,116,8.189095,80.527400,34.5572,2022-10-01,00:00:58
4,116,8.186705,80.527848,35.0972,2022-10-01,00:01:13


In [11]:
trip_data = pd.read_csv("../ALL_BUS/trip_all.csv")

In [12]:
trip_data = trip_data[(trip_data['date'] <= '2022-10-07') & (trip_data['date'] >= '2022-10-01')]

In [13]:
trip_data = trip_data[["device_id","date","start_time","start_terminal","end_time"]]

In [14]:
trip_data.head()

,device_id,date,start_time,start_terminal,end_time
1666,116,2022-10-01,08:41:39,BT02,09:35:36
1667,116,2022-10-01,09:51:51,BT01,10:42:47
1668,116,2022-10-01,13:42:11,BT02,14:30:32
1669,116,2022-10-01,15:09:08,BT01,16:00:26
1670,116,2022-10-02,07:42:11,BT02,08:22:32


In [15]:
# Preprocess 'date' and 'time' columns to ensure consistent format
trip_data['date'] = pd.to_datetime(trip_data['date'], errors='coerce', format='%Y-%m-%d')
trip_data['start_time'] = pd.to_datetime(trip_data['start_time'], errors='coerce', format='%H:%M:%S').dt.time
trip_data['end_time'] = pd.to_datetime(trip_data['end_time'], errors='coerce', format='%H:%M:%S').dt.time

processed_data['date'] = pd.to_datetime(processed_data['date'], errors='coerce', format='%Y-%m-%d')
processed_data['time'] = pd.to_datetime(processed_data['time'], errors='coerce', format='%H:%M:%S').dt.time


# Function to find start_time and start_terminal
def find_start_info(row):
    matching_rows = trip_data[(trip_data['device_id'] == row['deviceid']) & 
                        (trip_data['date'] == row['date']) & 
                        (row['time'] >= trip_data['start_time']) & 
                        (row['time'] <= trip_data['end_time'])]
    if not matching_rows.empty:
        return matching_rows.iloc[0]['start_time'], matching_rows.iloc[0]['start_terminal']
    else:
        return None, None

# Apply the function to the first dataset to find start_time and start_terminal
processed_data[['start_time', 'start_terminal']] = processed_data.apply(find_start_info, axis=1, result_type='expand')

# Print the updated DataFrame
print(processed_data)

        deviceid  latitude  longitude    speed       date      time  \
0            116  8.195475  80.525735  26.9979 2022-10-01  00:00:13   
1            116  8.193405  80.525917  30.2376 2022-10-01  00:00:28   
2            116  8.191373  80.526890  31.8575 2022-10-01  00:00:43   
3            116  8.189095  80.527400  34.5572 2022-10-01  00:00:58   
4            116  8.186705  80.527848  35.0972 2022-10-01  00:01:13   
...          ...       ...        ...      ...        ...       ...   
127994       123  7.296305  80.736062   0.0000 2022-10-06  23:53:59   
127995       250  7.283670  80.695938   0.0000 2022-10-06  23:55:06   
127996       121  7.296612  80.715932   0.0000 2022-10-06  23:56:37   
127997       275  7.292808  80.721027   0.0000 2022-10-06  23:58:41   
127998      1377  7.263868  80.700605   0.0000 2022-10-06  23:59:37   

       start_time start_terminal  
0            None           None  
1            None           None  
2            None           None  
3      

In [30]:
busStartTime = dict()

In [31]:
# #Function to find start time of each trip
# def findBusStartTime(row):
#     global busStartTime
 
#     if isOutsideBusStation(row['latitude'],row['longitude']):
#         if busStartTime.get(row['deviceid']) is not None:
#             return busStartTime.get(row['deviceid'])
#         else:
#             busStartTime[row['deviceid']]=row['devicetime']
#             return row['devicetime']

#     if row['deviceid'] in busStartTime:
#         busStartTime.pop(row['deviceid'])
#     return None 

In [32]:
processed_data.head()

,deviceid,latitude,longitude,speed,date,time,start_time,start_terminal,travel_time,dwell_time,SITR,hour_of_the_day,rush_hour
0,116,8.195475,80.525735,26.9979,2022-10-01,00:00:13,None,None,NaN,NaN,NaN,0,0
1,116,8.193405,80.525917,30.2376,2022-10-01,00:00:28,None,None,NaN,NaN,NaN,0,0
2,116,8.191373,80.526890,31.8575,2022-10-01,00:00:43,None,None,NaN,NaN,NaN,0,0
3,116,8.189095,80.527400,34.5572,2022-10-01,00:00:58,None,None,NaN,NaN,NaN,0,0
4,116,8.186705,80.527848,35.0972,2022-10-01,00:01:13,None,None,NaN,NaN,NaN,0,0


In [33]:
def timeDifference(t2,t1):

    date = datetime(2022, 1, 1)
    datetime1 = datetime.combine(date, t1)
    datetime2 = datetime.combine(date, t2)

    # Calculate the time difference in seconds
    time_difference_seconds = (datetime2 - datetime1).seconds

    return round((time_difference_seconds/60),2)

In [34]:
def findTravelTime(row):
    if row['start_time'] is not None:
        return timeDifference(row['time'],row['start_time'])
    return None

In [35]:
processed_data['travel_time'] = processed_data.apply(findTravelTime,axis=1)

In [36]:
busDwellTime = dict()
previousTimeStamp = dict()

In [37]:
def findDwellTime(row):
    global busDwellTime
    global previousTimeStamp
    
    if row['start_time'] is not None:
        if row['speed'] == 0:

            if busDwellTime.get(row['deviceid']) is not None:
                busDwellTime[row['deviceid']]=busDwellTime[row['deviceid']]+timeDifference(row['time'],previousTimeStamp[row['deviceid']])

            elif previousTimeStamp.get(row['deviceid']) is not None:
                busDwellTime[row['deviceid']]=timeDifference(row['time'],previousTimeStamp[row['deviceid']])
                
            else:
                busDwellTime[row['deviceid']]=0

        elif busDwellTime.get(row['deviceid']) is None:
            busDwellTime[row['deviceid']]=0
    
        previousTimeStamp[row['deviceid']] = row['time']
        return busDwellTime[row['deviceid']]

    else:
        if row['deviceid'] in busDwellTime:
            busDwellTime.pop(row['deviceid'])
        if row['deviceid'] in previousTimeStamp:
            previousTimeStamp.pop(row['deviceid'])
        return None

In [38]:
processed_data['dwell_time'] = processed_data.apply(findDwellTime,axis=1)

In [39]:
def findSITR(row):
    if row['travel_time'] is not None and row['travel_time']!=0:
        return round((row['dwell_time']/row['travel_time']),3)
    return 0

In [40]:
processed_data["SITR"] = processed_data.apply(findSITR,axis=1)

In [41]:
processed_data['hour_of_the_day'] = processed_data['time'].apply(lambda x: x.hour)

In [42]:
def calculate_rush_hour(row):
    if (0 <= row['hour_of_the_day'] <= 5) or (18 <= row['hour_of_the_day'] <= 23):
        return 0
    elif ((7 <= row['hour_of_the_day'] <= 8) and (row['start_terminal'] == 'BT02')):
        return 0
    elif (((7 <= row['hour_of_the_day'] <= 11) or (14 <= row['hour_of_the_day'] <= 17)) and (row['start_terminal'] == 'BT01')):
        return 1
    elif (((6 == row['hour_of_the_day']) or (12 <= row['hour_of_the_day'] <= 13)) and (row['start_terminal'] == 'BT01')):
        return 2
    elif ((row['hour_of_the_day'] == 13) and (row['start_terminal'] == 'BT02')):
        return 2
    elif (((6 == row['hour_of_the_day']) or (9 <= row['hour_of_the_day'] <= 12) or (14 <= row['hour_of_the_day'] <= 17)) and (row['start_terminal'] == 'BT02')):
        return 1
    else:
        return 0  # or any other default value, 0 indicates no rush hour

# Apply the function to create the 'rush_hour' column
processed_data['rush_hour'] = processed_data.apply(calculate_rush_hour, axis=1)


In [45]:
processed_data["date"].unique()

array(['2022-10-01', '2022-10-02', '2022-10-03', '2022-10-04',
       '2022-10-05', '2022-10-06'], dtype=object)

In [43]:
processed_data["date"] = processed_data["date"].dt.strftime('%Y-%m-%d')

AttributeError: Can only use .dt accessor with datetimelike values

In [46]:
bus_running_time_df = pd.read_csv("../../Raw GPS data Kandy Buses/bus_running_times_feature_added_all.csv")
bus_running_time_df = bus_running_time_df[(bus_running_time_df["date"]>='2022-10-01') & (bus_running_time_df["date"]<='2022-10-07')]
bus_running_time_df["hour_of_day"] = bus_running_time_df["hour_of_day"].astype(int)
bus_running_time_df.head()

,trip_id,deviceid,direction,segment,date,start_time,end_time,run_time,run_time_in_seconds,length,...,rt(n-2),rt(n-3),hour_of_day,day,month,temp,precip,windspeed,conditions,dt(n-1)
159839,23390.0,1166.0,1.0,1.0,2022-10-01,06:40:55,06:42:55,0:02:00,120.0,0.63,...,113.0,113.0,6,1.0,10.0,19.3,0.0,10.4,Partially cloudy,0.0
159840,23390.0,1166.0,1.0,2.0,2022-10-01,06:44:25,06:48:55,0:04:30,270.0,1.28,...,279.0,279.0,6,1.0,10.0,19.3,0.0,10.4,Partially cloudy,90.0
159841,23390.0,1166.0,1.0,3.0,2022-10-01,06:49:25,06:57:25,0:08:00,480.0,2.11,...,120.0,546.0,6,1.0,10.0,19.3,0.0,10.4,Partially cloudy,30.0
159842,23390.0,1166.0,1.0,4.0,2022-10-01,06:57:55,07:01:10,0:03:15,195.0,1.55,...,270.0,120.0,6,1.0,10.0,19.3,0.0,10.4,Partially cloudy,30.0
159843,23390.0,1166.0,1.0,5.0,2022-10-01,07:01:10,07:03:23,0:02:13,133.0,0.84,...,480.0,270.0,7,1.0,10.0,19.8,0.0,10.1,Overcast,0.0


In [47]:
def find_windSpeedandConditions(row):
    matching_rows = bus_running_time_df[(bus_running_time_df['date'] == row['date']) & 
                                        (bus_running_time_df['hour_of_day'] == row['hour_of_the_day'])]
    if not matching_rows.empty:
        return matching_rows['windspeed'].values[0], matching_rows['conditions'].values[0], matching_rows['weekday/end'].values[0]
    else:
        return None, None, None

processed_data[['wind_speed', 'weather', 'weekday']] = processed_data.apply(find_windSpeedandConditions, axis=1, result_type='expand')

print(processed_data)

        deviceid  latitude  longitude    speed        date      time  \
0            116  8.195475  80.525735  26.9979  2022-10-01  00:00:13   
1            116  8.193405  80.525917  30.2376  2022-10-01  00:00:28   
2            116  8.191373  80.526890  31.8575  2022-10-01  00:00:43   
3            116  8.189095  80.527400  34.5572  2022-10-01  00:00:58   
4            116  8.186705  80.527848  35.0972  2022-10-01  00:01:13   
...          ...       ...        ...      ...         ...       ...   
127994       123  7.296305  80.736062   0.0000  2022-10-06  23:53:59   
127995       250  7.283670  80.695938   0.0000  2022-10-06  23:55:06   
127996       121  7.296612  80.715932   0.0000  2022-10-06  23:56:37   
127997       275  7.292808  80.721027   0.0000  2022-10-06  23:58:41   
127998      1377  7.263868  80.700605   0.0000  2022-10-06  23:59:37   

       start_time start_terminal  travel_time  dwell_time  SITR  \
0            None           None          NaN         NaN   NaN   
1

In [48]:
processed_data = processed_data[(processed_data['SITR']<=1) | (pd.isnull(processed_data['SITR']))]
processed_data = processed_data[(processed_data['travel_time']<=90) | (pd.isnull(processed_data['SITR']))]
processed_data = processed_data[(processed_data['dwell_time']<=30) | (pd.isnull(processed_data['SITR']))]

In [49]:
processed_data = processed_data.dropna()

In [50]:
processed_data.isnull().sum()

deviceid           0
latitude           0
longitude          0
speed              0
date               0
time               0
start_time         0
start_terminal     0
travel_time        0
dwell_time         0
SITR               0
hour_of_the_day    0
rush_hour          0
wind_speed         0
weather            0
weekday            0
dtype: int64

In [52]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
processed_data['weather_encoded'] = label_encoder.fit_transform(processed_data['weather'])

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

columns_to_normalize = ['hour_of_the_day', 'rush_hour', 'wind_speed']

processed_data[columns_to_normalize] = scaler.fit_transform(processed_data[columns_to_normalize])

In [55]:
processed_data.head()

,deviceid,latitude,longitude,speed,date,time,start_time,start_terminal,travel_time,dwell_time,SITR,hour_of_the_day,rush_hour,wind_speed,weather,weekday,weather_encoded
1685,505,7.290552,80.641425,17.2786,2022-10-01,06:00:02,05:28:56,BT02,31.10,3.22,0.104,-1.713643,0.001922,0.069337,Partially cloudy,0.0,2
1688,505,7.290563,80.640318,16.7387,2022-10-01,06:00:17,05:28:56,BT02,31.35,3.22,0.103,-1.713643,0.001922,0.069337,Partially cloudy,0.0,2
1691,505,7.290412,80.639167,16.7387,2022-10-01,06:00:32,05:28:56,BT02,31.60,3.22,0.102,-1.713643,0.001922,0.069337,Partially cloudy,0.0,2
1694,505,7.290635,80.638265,16.7387,2022-10-01,06:00:47,05:28:56,BT02,31.85,3.22,0.101,-1.713643,0.001922,0.069337,Partially cloudy,0.0,2
1697,505,7.291268,80.637207,15.1188,2022-10-01,06:01:02,05:28:56,BT02,32.10,3.22,0.100,-1.713643,0.001922,0.069337,Partially cloudy,0.0,2


In [56]:
processed_data.to_csv('processed_data.csv',index=False)